In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
from model import Model
from dataloader import databunch_from_csv
from utils import threshold_predictions, check_acc
from fastai.vision import models

In [7]:
label_file = 'train_tags_combined.csv'

In [8]:
data = databunch_from_csv(csv_name=label_file, path=constants.dataset_folder,
                          img_size=constants.initial_dimension, vertical_flip=False,
                          batch_size=constants.batch_size)

## Building Model

The model built has `245` output nodes and penultimate activation function is `sigmoid` as opposed to `softmax` layer. The main reason behind this decision is allowing the model to make multiple predictions simultaneously and thresholding afterwards to get final prediction outputs. When it come to recognizing cars, allowing the model to make multiple predictions is beneficial.

ResNet50 is used as the model of choice. Pretrained weights on Imagenet are used as initial weights for convolutional layers.

In [9]:
class_names = data.classes
learn = Model(threshold=0.5, databunch=data, arch=models.resnet50)

## Training Model

Finding a good initial learning rate for freezed model (only the newly added layers are trained).

In [7]:
learn.learn.lr_find()
learn.learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


Model is trained with cyclical learning rate with early stopping.

In [9]:
learn.train_model(lr=5E-02, high_lr=None, epochs=15, freeze=True)

epoch,train_loss,valid_loss,accuracy_thresh,fbeta,time
0,0.031801,0.023893,0.993562,0.259852,02:32
1,0.020335,0.016100,0.995234,0.484102,02:32
2,0.018702,0.016265,0.995236,0.496305,02:37
3,0.020489,0.020538,0.994754,0.404261,02:38
4,0.018314,0.015456,0.995926,0.566320,02:37
5,0.016739,0.013205,0.996064,0.627008,02:34
6,0.014885,0.012925,0.996577,0.633382,02:31
7,0.012621,0.009963,0.997037,0.673629,02:31
8,0.010761,0.007421,0.997670,0.775109,02:30
9,0.009033,0.006324,0.998011,0.788081,02:33


Epoch 10: early stopping


In [ ]:
learn.learn.save('cars_model_stage1')

Unfreeze the model, find a new learning rate, 

In [11]:
learn.learn.unfreeze()
learn.learn.lr_find()
learn.learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [13]:
learn.train_model(lr=3e-06, high_lr=5E-03, epochs=15, freeze=False)

epoch,train_loss,valid_loss,accuracy_thresh,fbeta,time
0,0.005681,0.004207,0.998590,0.855512,02:44
1,0.005148,0.003802,0.998706,0.867022,02:44
2,0.004855,0.003609,0.998788,0.873065,02:43
3,0.004118,0.003247,0.998908,0.888105,02:43
4,0.003827,0.003023,0.998966,0.895771,02:42


Epoch 5: early stopping


In [19]:
learn.learn.save('cars_model_stage2')